In [29]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import unix_timestamp, hour
import matplotlib.pyplot as plt
import pandas as pd
import time

spark = SparkSession.builder.master('local[*]').getOrCreate()


sc = spark.sparkContext

spark

In [30]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

Your runtime has 11.6 gigabytes of available RAM



In [31]:
# Comenzar a medir el tiempo
start_time = time.time()
# Cargar los datos
df = spark.read.format("parquet").option("inferSchema", "true").option("timestampFormat","yyyy-MM-dd HH:mm:ss").option("header", "true").option("mode", "DROPMALFORMED").load("yellow_tripdata_2017-01.parquet")
num_viajes = df.count()
num_viajes = df.count()
print(f"Cantidad de datos procesados: {num_viajes} viajes.")

Cantidad de datos procesados: 9710820 viajes.


In [32]:
df_limited = df["tip_amount","DOLocationID"]

In [24]:
# Agrupar por 'DOLocationID' y calcular la media de 'tip_amount'
df_avg_tips = df_limited.groupBy("DOLocationID").agg({"tip_amount": "avg"})

# Renombrar columnas para mayor claridad
df_avg_tips = df_avg_tips.withColumnRenamed("avg(tip_amount)", "average_tip")

# Si quieres ordenar por la media de propina de mayor a menor:
df_avg_tips_ordered = df_avg_tips.orderBy("average_tip", ascending=False)

df_avg_tips_ordered.show()

# Finalizar medición del tiempo
end_time = time.time()
execution_time = end_time - start_time
print(f"Tiempo de ejecución: {execution_time:.2f} segundos.")


+------------+------------------+
|DOLocationID|       average_tip|
+------------+------------------+
|         204|15.657611940298507|
|          44|12.607241379310345|
|           2|12.126249999999999|
|           1|10.954543615676386|
|         109| 10.73825581395349|
|         176|10.382933333333337|
|          84| 9.383253012048192|
|           5| 9.147704918032787|
|         265| 8.874212609556167|
|         214| 8.320777777777776|
|         245| 8.035326086956523|
|          23| 7.677205240174674|
|         172| 7.591983471074382|
|         251| 7.571948051948051|
|         118| 7.352035928143712|
|         206|7.0175956284152985|
|         132| 6.725178089023828|
|           6| 6.120670103092784|
|         156| 5.911585365853658|
|         138|  5.85998675301736|
+------------+------------------+
only showing top 20 rows

Tiempo de ejecución: 7.69 segundos.
